## Entrenamiento completo
Hemos visto los diferentes componentes y pre requisitos de código para comenzar a entrenar nuestras redes neuronales. Ahora pondremos todos los aspectos juntos. El entrenamiento de un modelo es un proceso iterativo. En cada iteración se debe seguir los siguientes pasos:

1. Recolectar muestras
2. hacer una predicción
3. calcular el error de la predicción (loss)
4. calcular la gradiente del error con respecto de los parámetros
5. modificar los parámetros de acuerdo a un criterio de **optimización**


In [1]:
# codigo visto previamente
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor



## Carga del dataset

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

## Definición del modelo de la red neuronal

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

# Hiperparámetros

Los hiperparámetros son parámetros ajustables que nos permiten controlar el proceso de optimización y entrenamiento del modelo. La variación en los valores de los hiperparámetros impactan en el entrenamiento y los radios de convergencia. Definiremos los siguientes hiperparámetros para este ejemplo:

- Número de Épocas: El número de veces de iterar sobre todo el dataset de entrenamiento.
- Batch Size: Tamaño del minilote. El número de muestras propagadas a través de la red antes de ser actualizados los parámetros de la red.
- Learning Rate: Un parámetro de cantidad de variación de parámetros en cada lote/época. Valores pequeños hacen un entrenamiento lento, valores muy grandes pueden hacer el entrenamiento inestable.

In [4]:
learning_rate = 1e-3
batch_size = 64
epochs = 10

# Bucle de optimización

Una vez tenemos los dataset, definición del modelo y los hiperparámetros, podemos proceder a correr el bucle de entrenamiento. Cada iteración del bucle se conoce como Época. En cada época realizamos 2 procesos:

1. Train Loop: El bucle de entrenamiento y optimización de los parámetros de la red (Backpropagation).
2. Validation/Test Loop: Bucle para recolectar métricas de rendimiento durante el entrenamiento del modelo.

### Función de costo
Las funciones de costo miden el grado de similitud entre las predicciones de la red y los resultados esperados. La idea principal del entrenamiento de una red es minimizar la función de costo. 

Pytorch incluye implementaciones de funciones de costo comunes como MSE, NLL, CrossEntropy y otros.

In [5]:
loss_fn = nn.CrossEntropyLoss()

### Optimizador
La optimizaci'on es el proceso de ajustar los parámetros del modelo para reducir el error en cada paso de entrenamiento. Los algoritmos de optimización u Optimizadores definen la forma en la que este proceso es realizado. Toda la lógica de optimización esta encapsulada en el objeto `optimizer`. 

Pytorch también incluye implementaciones de optimizadores populares como ADAM, RMSProp, SGD y otros.

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### Implemetanción completa

In [7]:
# bucle de entrenamiento

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # modo entrenamiento
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # forward
        # prediccion y costo
        pred = model(X)
        loss = loss_fn(pred, y)

        # backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# funcion de pruebas
def test_loop(dataloader, model, loss_fn):
    # modo evaluacion
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    # evaluacion del modelo con torch.no_grad
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")



In [8]:
# bucle principal
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.308186  [   64/60000]
loss: 2.291214  [ 6464/60000]
loss: 2.275085  [12864/60000]
loss: 2.264726  [19264/60000]
loss: 2.250510  [25664/60000]
loss: 2.221443  [32064/60000]
loss: 2.228258  [38464/60000]
loss: 2.193028  [44864/60000]
loss: 2.188263  [51264/60000]
loss: 2.149150  [57664/60000]
Test Error: 
 Accuracy: 37.4%, Avg loss: 2.151133 

Epoch 2
-------------------------------
loss: 2.162347  [   64/60000]
loss: 2.154422  [ 6464/60000]
loss: 2.095660  [12864/60000]
loss: 2.107693  [19264/60000]
loss: 2.062939  [25664/60000]
loss: 1.998299  [32064/60000]
loss: 2.018794  [38464/60000]
loss: 1.936786  [44864/60000]
loss: 1.940431  [51264/60000]
loss: 1.863470  [57664/60000]
Test Error: 
 Accuracy: 60.3%, Avg loss: 1.867706 

Epoch 3
-------------------------------
loss: 1.901198  [   64/60000]
loss: 1.878707  [ 6464/60000]
loss: 1.754277  [12864/60000]
loss: 1.789234  [19264/60000]
loss: 1.690365  [25664/60000]
loss: 1.638416  [32064/600